## Set up `GenX` Project in Current Folder

This step assumes you've installed `julia` already but have not set up the current `GenX` project folder.

In [ ]:
!julia --project=. Install.jl

## Export `GenX` Input CSVs from Spreadsheet

In [ ]:
import shutil

import xlwings as xw
import pandas as pd
from collections import defaultdict
import os
from upath import UPath
from loguru import logger
import sys
from datetime import datetime

logger.remove()
logger.add(sys.stderr, backtrace=False)

- [x] Do the xlwings thing where the SharePoint path is updated in the spreadsheet -> used VBA UDF
- [ ] Do we need something like `kit-ui connect`? I don't think so
- [ ] File picker to connect to spreadsheet?
- [x] Format this notebook before committing
- [x] Copy Run.jl
- [x] Copy settings
- [ ] Check that demand_data, fuel_data, and generator_variability CSVs have the same length
- [ ] Save a mapping of the planning period to year, so that it can be read in by spreadsheet, or when concatenating the dataframe (instead of guessing)

In [ ]:
wb = xw.Book(
    '/Users/roderick/Library/CloudStorage/OneDrive-SharedLibraries-ResilientTransition/5.001 Kentucky Resource Council - Documents/Data/Kentucky Load Resource Model.xlsb'
)

In [ ]:
def save_case(base_folder: UPath, case_subfolder: str | None = None):
    global name, range, col
    # Get CSV names as a nested dictionary (since some CSVs have been split into multiple separate tables
    # Named ranges have the format of [csv file name]...[#]...[optional transformation, either .T or .ffill]
    csv_names = defaultdict(list)
    for name in wb.names:
        if ".csv" in name.name:
            csv_names[name.name.split("...")[0]].append(name)
    for csv_name, ranges in csv_names.items():
        dfs = []
        for rng in ranges:
            # Get each range as a dataframe
            df = rng.refers_to_range.options(pd.DataFrame, index=0,
                                               header=(1 if not rng.name.endswith("...T") else 0)).value
            df = df.dropna(how="all", axis=1)
            df = df.dropna(how="all", axis=0)
            if "resource" in df.columns:
                df = df.dropna(subset="resource", axis=0)
            if "drop" in df.columns:
                df = df[df["drop"] != True]

            # Apply optional transform
            if rng.name.endswith("...T"):
                df = df.set_index(df.columns[0])
                df = df.T
            elif rng.name.endswith("...ffill"):
                df = df.ffill()
            elif rng.name.endswith("...drop...1"):
                df = df.iloc[:, [0, -1]]
                df = df.dropna(how="any")
            elif rng.name.endswith("...drop...3"):
                df = df.iloc[:, [0, -3, -2, -1]]
                df = df.dropna(how="any")

            if csv_name in [
                "resources\\policy_assignments\\Resource_NQC_derate.csv",
                "resources\\policy_assignments\\ELCC_multipliers.csv",
                "resources\\Resource_multistage_data.csv",
            ]:
                df = df.rename(columns={"resource": "Resource"})

            # Change types for columns to int & strings
            int_columns = [
                col for col in df.columns if col in
                 [
                     "can_retire",
                     "zone",
                     "new_build",
                     "model",
                     "lds",
                     "Time_Index"
                 ]
            ]
            df[int_columns] = df[int_columns].astype(int)

            str_columns = [
                col for col in df.columns if col in
                 [
                     "cluster",
                     "region",
                 ]
            ]
            df[str_columns] = df[str_columns].astype(str)

            if df.isna().any().any():
                logger.error(f"{csv_name} has blank cells. GenX currently does not have consistent handling of missing data, so please fill in or add placeholder values.")

            dfs.append(df)

        # Join all the dfs
        final_df = pd.concat([df.reset_index(drop=True) for df in dfs], axis=1)

        # Save joined dataframe to CSV
        planning_period_folder = base_folder / case_subfolder if case_subfolder else base_folder
        filepath = planning_period_folder / csv_name.replace("\\", os.sep)
        filepath.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_csv(filepath, index=False)

In [ ]:
base_folder = UPath(wb.names["BaseFolder"].refers_to_range.value)
case_name = wb.names["CaseName"].refers_to_range.value

if base_folder.exists():
    logger.warning(f"Overwriting case: {base_folder}")

planning_periods = wb.sheets["GenX Settings"].tables["ModeledYears"].range.options(pd.DataFrame, index=1).value.dropna().index.astype(int).values

counter = 1
for planning_period in planning_periods:
    wb.sheets["GenX Settings"].range("ActiveYear").value = planning_period
    wb.app.calculate()

    logger.info(f"Saving case inputs for {planning_period}: (inputs_p{counter})")
    save_case(base_folder=base_folder, case_subfolder=f"inputs/inputs_p{counter}")
    counter += 1

    # Save settings .yml files
    wb.sheets["GenX Settings"].range("settings\genx_settings.yml").options(pd.DataFrame).value

# Settings
logger.info("Saving settings...")

base_settings_folder = UPath("/Users/roderick/PycharmProjects/resilient-transition/GenX.jl/__base_settings__")

if (base_folder / "settings").exists():
    shutil.rmtree(base_folder / "settings")
shutil.copytree(base_settings_folder, base_folder / "settings")

wb.sheets["GenX Settings"].range(r"settings\genx_settings.yml").options(pd.Series, header=False).value.astype(int).reset_index().astype(str).agg("".join, axis=1).to_csv(base_folder / "settings" / "genx_settings.yml", index=False, header=False, sep="\t")

wb.sheets["GenX Settings"].range(r"settings\multi_stage_settings.yml").options(pd.Series, header=False).value.apply(lambda x: int(x) if isinstance(x, (float, bool, int)) else x).reset_index().astype(str).agg("".join, axis=1).to_csv(base_folder / "settings" / "multi_stage_settings.yml", index=False, header=False, sep="\t")

wb.sheets["GenX Settings"].range(r"settings\time_domain_reduction_settings.yml").options(pd.Series, header=False).value.replace({None: " "}).apply(lambda x: int(x) if isinstance(x, (float, bool, int)) else x).reset_index().astype(str).agg("".join, axis=1).replace({"None": ""}).to_csv(base_folder / "settings" / "time_domain_reduction_settings.yml", index=False, header=False, sep="\t")

logger.success(f"Saved multi-stage capacity expansion case: {case_name}")

## Running Case

In [ ]:
!julia --project=. Run.jl $base_folder

In [ ]:
# Total Capacity
portfolio = pd.read_csv(base_folder / "results" / "capacities_multi_stage.csv", index_col=0)
portfolio = portfolio[[col for col in portfolio.columns if not col.startswith("StartCap")]]
wb.sheets["GenX Results"].range("capacities_multi_stage").value = portfolio

# Builds
periods_range = range(1, len(list((base_folder / "results").glob("results_p*"))) + 1)


builds = pd.concat([pd.read_csv(base_folder / "results" / f"results_p{p}" / "capacity.csv", index_col=0)["NewCap"] for p in periods_range], axis=1)
wb.sheets["GenX Results"].range("capacities").value = builds

# CFs
cfs = pd.concat([pd.read_csv(base_folder / "results" / f"results_p{p}" / "capacityfactor.csv", index_col=0)["CapacityFactor"] for p in periods_range], axis=1)
wb.sheets["GenX Results"].range("cfs").value = cfs


wb.sheets["GenX Results"].activate()
wb.app.calculate()
print(f"Loaded results at: {datetime.now()}")